# Word2Vec Tutorial
This tutorial follows a [blog post](http://rare-technologies.com/word2vec-tutorial/) written by the creator of gensim.

## Preparing the Input
Starting from the beginning, gensim’s `word2vec` expects a sequence of sentences as its input. Each sentence a list of words (utf8 strings):

In [1]:
# import modules & set up logging
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
sentences = [['first', 'sentence'], ['second', 'sentence']]
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=1)

2016-11-10 19:28:19,521 : INFO : collecting all words and their counts
2016-11-10 19:28:19,525 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-11-10 19:28:19,525 : INFO : collected 3 word types from a corpus of 4 raw words and 2 sentences
2016-11-10 19:28:19,529 : INFO : min_count=1 retains 3 unique words (drops 0)
2016-11-10 19:28:19,533 : INFO : min_count leaves 4 word corpus (100% of original 4)
2016-11-10 19:28:19,533 : INFO : deleting the raw counts dictionary of 3 items
2016-11-10 19:28:19,533 : INFO : sample=0.001 downsamples 3 most-common words
2016-11-10 19:28:19,537 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2016-11-10 19:28:19,537 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2016-11-10 19:28:19,542 : INFO : resetting layer weights
2016-11-10 19:28:19,542 : INFO : training model with 3 workers on 3 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5
2016-11-10 19:28:1

Keeping the input as a Python built-in list is convenient, but can use up a lot of RAM when the input is large.

Gensim only requires that the input must provide sentences sequentially, when iterated over. No need to keep everything in RAM: we can provide one sentence, process it, forget it, load another sentence…

For example, if our input is strewn across several files on disk, with one sentence per line, then instead of loading everything into an in-memory list, we can process the input file by file, line by line:

In [3]:
# create some toy data to use with the following example
import smart_open, os

if not os.path.exists('./data/'):
    os.makedirs('./data/')

filenames = ['./data/f1.txt', './data/f2.txt']

for i, fname in enumerate(filenames):
    with smart_open.smart_open(fname, 'w') as fout:
        for line in sentences[i]:
            fout.write(line + '\n')

In [4]:
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()

In [5]:
sentences = MySentences('./data/') # a memory-friendly iterator
print(list(sentences))

[['first'], ['sentence'], ['second'], ['sentence']]


In [6]:
# generate the Word2Vec model
model = gensim.models.Word2Vec(sentences, min_count=1)
print(model)
print(model.vocab)

2016-11-10 19:28:20,269 : INFO : collecting all words and their counts
2016-11-10 19:28:20,271 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-11-10 19:28:20,274 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2016-11-10 19:28:20,275 : INFO : min_count=1 retains 3 unique words (drops 0)
2016-11-10 19:28:20,275 : INFO : min_count leaves 4 word corpus (100% of original 4)
2016-11-10 19:28:20,279 : INFO : deleting the raw counts dictionary of 3 items
2016-11-10 19:28:20,280 : INFO : sample=0.001 downsamples 3 most-common words
2016-11-10 19:28:20,280 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2016-11-10 19:28:20,280 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2016-11-10 19:28:20,280 : INFO : resetting layer weights
2016-11-10 19:28:20,280 : INFO : training model with 3 workers on 3 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5
2016-11-10 19:28:2

Word2Vec(vocab=3, size=100, alpha=0.025)
{'second': <gensim.models.word2vec.Vocab object at 0x000001E111E39390>, 'sentence': <gensim.models.word2vec.Vocab object at 0x000001E111E39320>, 'first': <gensim.models.word2vec.Vocab object at 0x000001E111E39438>}


Say we want to further preprocess the words from the files — convert to unicode, lowercase, remove numbers, extract named entities… All of this can be done inside the `MySentences` iterator and `word2vec` doesn’t need to know. All that is required is that the input yields one sentence (list of utf8 words) after another.

**Note to advanced users:** calling `Word2Vec(sentences)` will run two passes over the sentences iterator. 
  1. The first pass collects words and their frequencies to build an internal dictionary tree structure. 
  2. The second pass trains the neural model.

These two passes can also be initiated manually, in case your input stream is non-repeatable (you can only afford one pass), and you’re able to initialize the vocabulary some other way:

In [7]:
# build the same model, making the 2 steps explicit
new_model = gensim.models.Word2Vec(min_count=1)  # an empty model, no training
new_model.build_vocab(sentences)                 # can be a non-repeatable, 1-pass generator     
new_model.train(sentences)                       # can be a non-repeatable, 1-pass generator
print(new_model)
print(model.vocab)

2016-11-10 19:28:20,410 : INFO : collecting all words and their counts
2016-11-10 19:28:20,414 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-11-10 19:28:20,414 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2016-11-10 19:28:20,414 : INFO : min_count=1 retains 3 unique words (drops 0)
2016-11-10 19:28:20,414 : INFO : min_count leaves 4 word corpus (100% of original 4)
2016-11-10 19:28:20,414 : INFO : deleting the raw counts dictionary of 3 items
2016-11-10 19:28:20,414 : INFO : sample=0.001 downsamples 3 most-common words
2016-11-10 19:28:20,414 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2016-11-10 19:28:20,414 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2016-11-10 19:28:20,430 : INFO : resetting layer weights
2016-11-10 19:28:20,430 : INFO : training model with 3 workers on 3 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5
2016-11-10 19:28:2

Word2Vec(vocab=3, size=100, alpha=0.025)
{'second': <gensim.models.word2vec.Vocab object at 0x000001E111E39390>, 'sentence': <gensim.models.word2vec.Vocab object at 0x000001E111E39320>, 'first': <gensim.models.word2vec.Vocab object at 0x000001E111E39438>}


## More data would be nice
For the following examples, we'll use the Lee Corpus (which you already have if you've installed gensim):

In [8]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data']) + os.sep
lee_train_file = test_data_dir + 'lee_background.cor'

In [9]:
class MyText(object):
    def __iter__(self):
        for line in open(lee_train_file):
            # assume there's one document per line, tokens separated by whitespace
            yield line.lower().split()

sentences = MyText()

print(sentences)

## Training
`Word2Vec` accepts several parameters that affect both training speed and quality.

One of them is for pruning the internal dictionary. Words that appear only once or twice in a billion-word corpus are probably uninteresting typos and garbage. In addition, there’s not enough data to make any meaningful training on those words, so it’s best to ignore them:

In [10]:
# default value of min_count=5
model = gensim.models.Word2Vec(sentences, min_count=10)

2016-11-10 19:28:21,077 : INFO : collecting all words and their counts
2016-11-10 19:28:21,081 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-11-10 19:28:21,133 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2016-11-10 19:28:21,149 : INFO : min_count=10 retains 806 unique words (drops 9380)
2016-11-10 19:28:21,153 : INFO : min_count leaves 40964 word corpus (68% of original 59890)
2016-11-10 19:28:21,176 : INFO : deleting the raw counts dictionary of 10186 items
2016-11-10 19:28:21,189 : INFO : sample=0.001 downsamples 54 most-common words
2016-11-10 19:28:21,193 : INFO : downsampling leaves estimated 26224 word corpus (64.0% of prior 40964)
2016-11-10 19:28:21,197 : INFO : estimated required memory for 806 words and 100 dimensions: 1047800 bytes
2016-11-10 19:28:21,209 : INFO : resetting layer weights
2016-11-10 19:28:21,253 : INFO : training model with 3 workers on 806 vocabulary and 100 features, using sg=0 hs=

In [11]:
# default value of size=100
model = gensim.models.Word2Vec(sentences, size=200)

2016-11-10 19:28:21,830 : INFO : collecting all words and their counts
2016-11-10 19:28:21,832 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-11-10 19:28:21,885 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2016-11-10 19:28:21,909 : INFO : min_count=5 retains 1723 unique words (drops 8463)
2016-11-10 19:28:21,910 : INFO : min_count leaves 46858 word corpus (78% of original 59890)
2016-11-10 19:28:21,927 : INFO : deleting the raw counts dictionary of 10186 items
2016-11-10 19:28:21,931 : INFO : sample=0.001 downsamples 49 most-common words
2016-11-10 19:28:21,934 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2016-11-10 19:28:21,939 : INFO : estimated required memory for 1723 words and 200 dimensions: 3618300 bytes
2016-11-10 19:28:21,960 : INFO : resetting layer weights
2016-11-10 19:28:22,096 : INFO : training model with 3 workers on 1723 vocabulary and 200 features, using sg=0 h

Bigger size values require more training data, but can lead to better (more accurate) models. Reasonable values are in the tens to hundreds.

The last of the major parameters (full list [here](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)) is for training parallelization, to speed up training:

In [12]:
# default value of workers=3 (tutorial says 1...)
model = gensim.models.Word2Vec(sentences, workers=4)

2016-11-10 19:28:22,730 : INFO : collecting all words and their counts
2016-11-10 19:28:22,740 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-11-10 19:28:22,802 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2016-11-10 19:28:22,834 : INFO : min_count=5 retains 1723 unique words (drops 8463)
2016-11-10 19:28:22,839 : INFO : min_count leaves 46858 word corpus (78% of original 59890)
2016-11-10 19:28:22,868 : INFO : deleting the raw counts dictionary of 10186 items
2016-11-10 19:28:22,871 : INFO : sample=0.001 downsamples 49 most-common words
2016-11-10 19:28:22,873 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2016-11-10 19:28:22,876 : INFO : estimated required memory for 1723 words and 100 dimensions: 2239900 bytes
2016-11-10 19:28:22,914 : INFO : resetting layer weights
2016-11-10 19:28:23,059 : INFO : training model with 4 workers on 1723 vocabulary and 100 features, using sg=0 h

The `workers` parameter only has an effect if you have [Cython](http://cython.org/) installed. Without Cython, you’ll only be able to use one core because of the [GIL](https://wiki.python.org/moin/GlobalInterpreterLock) (and `word2vec` training will be [miserably slow](http://rare-technologies.com/word2vec-in-python-part-two-optimizing/)).

## Memory
At its core, `word2vec` model parameters are stored as matrices (NumPy arrays). Each array is **#vocabulary** (controlled by min_count parameter) times **#size** (size parameter) of floats (single precision aka 4 bytes).

Three such matrices are held in RAM (work is underway to reduce that number to two, or even one). So if your input contains 100,000 unique words, and you asked for layer `size=200`, the model will require approx. `100,000*200*4*3 bytes = ~229MB`.

There’s a little extra memory needed for storing the vocabulary tree (100,000 words would take a few megabytes), but unless your words are extremely loooong strings, memory footprint will be dominated by the three matrices above.

## Evaluating
`Word2Vec` training is an unsupervised task, there’s no good way to objectively evaluate the result. Evaluation depends on your end application.

Google have released their testing set of about 20,000 syntactic and semantic test examples, following the “A is to B as C is to D” task. You can download a zip file [here](https://storage.googleapis.com/google-code-archive-source/v2/code.google.com/word2vec/source-archive.zip), and unzip it, to get the `questions-words.txt` file used below.

Gensim support the same evaluation set, in exactly the same format:

In [13]:
try:
    model.accuracy('./datasets/questions-words.txt')
except FileNotFoundError:
    raise ValueError("SKIP: please download the questions-word.txt file.")

2016-11-10 19:28:23,910 : INFO : precomputing L2-norms of word weight vectors
2016-11-10 19:28:23,924 : INFO : family: 0.0% (0/2)
2016-11-10 19:28:24,024 : INFO : gram3-comparative: 0.0% (0/12)
2016-11-10 19:28:24,060 : INFO : gram4-superlative: 0.0% (0/12)
2016-11-10 19:28:24,123 : INFO : gram5-present-participle: 0.0% (0/20)
2016-11-10 19:28:24,173 : INFO : gram6-nationality-adjective: 0.0% (0/20)
2016-11-10 19:28:24,233 : INFO : gram7-past-tense: 0.0% (0/20)
2016-11-10 19:28:24,267 : INFO : gram8-plural: 0.0% (0/12)
2016-11-10 19:28:24,282 : INFO : total: 0.0% (0/98)


This `accuracy` takes an 
[optional parameter](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.accuracy) `restrict_vocab` 
which limits which test examples are to be considered.

Once again, **good performance on this test set doesn’t mean word2vec will work well in your application, or vice versa**. It’s always best to evaluate directly on your intended task.

## Storing and loading models
You can store/load models using the standard gensim methods:

In [14]:
from tempfile import mkstemp

fs, temp_path = mkstemp("gensim_temp")  # creates a temp file

model.save(temp_path)  # save the model
new_model = gensim.models.Word2Vec.load(temp_path)  # open the model

2016-11-10 19:28:24,311 : INFO : saving Word2Vec object under C:\Users\vtush\AppData\Local\Temp\tmpp36mst2qgensim_temp, separately None
2016-11-10 19:28:24,320 : INFO : not storing attribute syn0norm
2016-11-10 19:28:24,322 : INFO : not storing attribute cum_table
2016-11-10 19:28:24,366 : INFO : loading Word2Vec object from C:\Users\vtush\AppData\Local\Temp\tmpp36mst2qgensim_temp
2016-11-10 19:28:24,397 : INFO : setting ignored attribute syn0norm to None
2016-11-10 19:28:24,398 : INFO : setting ignored attribute cum_table to None


which uses pickle internally, optionally `mmap`‘ing the model’s internal large NumPy matrices into virtual memory directly from disk files, for inter-process memory sharing.

In addition, you can load models created by the original C tool, both using its text and binary formats:

    model = gensim.models.Word2Vec.load_word2vec_format('/tmp/vectors.txt', binary=False)
    # using gzipped/bz2 input works too, no need to unzip:
    model = gensim.models.Word2Vec.load_word2vec_format('/tmp/vectors.bin.gz', binary=True)

## Online training / Resuming training
Advanced users can load a model and continue training it with more sentences:

In [15]:
model = gensim.models.Word2Vec.load(temp_path)
more_sentences = ['Advanced', 'users', 'can', 'load', 'a', 'model', 'and', 'continue', 
                  'training', 'it', 'with', 'more', 'sentences']
model.train(more_sentences)

# cleaning up temp
os.close(fs)
os.remove(temp_path)

2016-11-10 19:28:24,507 : INFO : loading Word2Vec object from C:\Users\vtush\AppData\Local\Temp\tmpp36mst2qgensim_temp
2016-11-10 19:28:24,615 : INFO : setting ignored attribute syn0norm to None
2016-11-10 19:28:24,620 : INFO : setting ignored attribute cum_table to None
2016-11-10 19:28:24,637 : INFO : training model with 4 workers on 1723 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5
2016-11-10 19:28:24,640 : INFO : expecting 300 sentences, matching count from corpus used for vocabulary survey
2016-11-10 19:28:24,650 : INFO : worker thread finished; awaiting finish of 3 more threads
2016-11-10 19:28:24,654 : INFO : worker thread finished; awaiting finish of 2 more threads
2016-11-10 19:28:24,659 : INFO : worker thread finished; awaiting finish of 1 more threads
2016-11-10 19:28:24,663 : INFO : worker thread finished; awaiting finish of 0 more threads
2016-11-10 19:28:24,666 : INFO : training on 320 raw words (36 effective words) took 0.0s, 2052 effective words/

You may need to tweak the `total_words` parameter to `train()`, depending on what learning rate decay you want to simulate.

Note that it’s not possible to resume training with models generated by the C tool, `load_word2vec_format()`. You can still use them for querying/similarity, but information vital for training (the vocab tree) is missing there.

## Using the model
`Word2Vec` supports several word similarity tasks out of the box:

In [16]:
model.most_similar(positive=['human', 'crime'], negative=['party'], topn=1)

2016-11-10 19:28:24,739 : INFO : precomputing L2-norms of word weight vectors


[('far', 0.9905146360397339)]

In [17]:
model.doesnt_match("input is lunch he sentence cat".split())

'sentence'

In [18]:
print(model.similarity('human', 'party'))
print(model.similarity('tree', 'murder'))

0.999397021779
0.994639666296


If you need the raw output vectors in your application, you can access these either on a word-by-word basis:

In [19]:
model['tree']  # raw NumPy vector of a word

array([ 0.0129236 , -0.04627566, -0.01077281,  0.02019315, -0.04912261,
       -0.01477702,  0.03356161,  0.00266525,  0.02809421,  0.01202157,
       -0.03432563, -0.01136982, -0.0103232 , -0.06312485, -0.00771275,
        0.012601  ,  0.05340691,  0.0548443 , -0.01060832,  0.01924257,
       -0.07651207, -0.00080719,  0.01463398, -0.01514605, -0.01436063,
       -0.04011078,  0.02258537,  0.03720149,  0.02813212,  0.00419395,
       -0.01009519, -0.06079235,  0.06957994, -0.04331882,  0.00960023,
       -0.0071871 ,  0.08013803, -0.06772202, -0.01903094, -0.00066973,
       -0.06936473, -0.00116428,  0.03776711, -0.00531811, -0.00416031,
        0.04436193,  0.02074978,  0.01759907, -0.04600459,  0.07960982,
        0.07946361, -0.02700096, -0.04801841, -0.04432948,  0.03373136,
        0.05676608, -0.02725234, -0.05159596,  0.01617298,  0.08130913,
       -0.02067144,  0.06205077, -0.00053941,  0.03779722, -0.02158519,
       -0.0324785 , -0.0363324 , -0.01082932,  0.0451992 , -0.04

…or en-masse as a 2D NumPy matrix from `model.syn0`.

## Outro
There is a **Bonus App** on the original [blog post](http://rare-technologies.com/word2vec-tutorial/), which runs `word2vec` on the Google News dataset, of **about 100 billion words**.

Full `word2vec` API docs [here](http://radimrehurek.com/gensim/models/word2vec.html); get [gensim](http://radimrehurek.com/gensim/) here. Original C toolkit and `word2vec` papers by Google [here](https://code.google.com/archive/p/word2vec/).